In [ ]:
# installiamo le librerie necessarie
!pip install --upgrade --quiet langchain
!pip install --upgrade --quiet langchain_openai
!pip install --upgrade --quiet langchain_chroma
!pip install --upgrade --quiet langchain_community

In [3]:
# Carichiamo l'API Key di OpenAI
import os
os.environ['OPENAI_API_KEY'] = 'xxx'

In [4]:
# Usiamo il Document Loader per caricare il video
from langchain_community.document_loaders import TextLoader
loader = TextLoader("./i_promessi_sposi.txt")
documents = loader.load()

# il nostro documento è ora pronto per essere analizzato. Ma vediamo prima cosa c'è dentro
print(documents)

[Document(metadata={'source': './i_promessi_sposi.txt'}, page_content="                           I PROMESSI SPOSI\n                            CAPITOLO PRIMO.\nQuel ramo del lago di Como, che volge a mezzogiorno, tra due catene\nnon interrotte di monti, tutte a seni e a golfi, a seconda dello\nsporgere e del rientrare di quelli, vien, quasi a un tratto, a\nristringersi, e a prender corso e figura di fiume, tra un promontorio\na destra, e un'ampia costiera dall'altra parte; e il ponte, che ivi\ncongiunge le due rive, par che renda ancor più sensibile all'occhio\nquesta trasformazione, e segni il punto in cui il lago cessa, e\nl'Adda rincomincia, per ripigliar poi nome di lago dove le rive,\nallontanandosi di nuovo, lascian l'acqua distendersi e rallentarsi in\nnuovi golfi e in nuovi seni. La costiera, formata dal deposito di tre\ngrossi torrenti, scende appoggiata a due monti contigui, l'uno detto di\nsan Martino, l'altro, con voce lombarda, il _Resegone_, dai molti suoi\ncocuzzoli in 

In [5]:
# Il nostro documento è troppo grande per entrare nella context windows di GPT
# Usiamo un text spliltter per dividerlo in documenti più piccoli
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=5000,
)
documents = text_splitter.split_documents(documents)
print(f"Diviso il testo in {len(documents)} documenti")
print(documents)

Divise le trascrizioni in 271 documenti
[Document(metadata={'source': './i_promessi_sposi.txt'}, page_content="I PROMESSI SPOSI\n                            CAPITOLO PRIMO.\nQuel ramo del lago di Como, che volge a mezzogiorno, tra due catene\nnon interrotte di monti, tutte a seni e a golfi, a seconda dello\nsporgere e del rientrare di quelli, vien, quasi a un tratto, a\nristringersi, e a prender corso e figura di fiume, tra un promontorio\na destra, e un'ampia costiera dall'altra parte; e il ponte, che ivi\ncongiunge le due rive, par che renda ancor più sensibile all'occhio\nquesta trasformazione, e segni il punto in cui il lago cessa, e\nl'Adda rincomincia, per ripigliar poi nome di lago dove le rive,\nallontanandosi di nuovo, lascian l'acqua distendersi e rallentarsi in\nnuovi golfi e in nuovi seni. La costiera, formata dal deposito di tre\ngrossi torrenti, scende appoggiata a due monti contigui, l'uno detto di\nsan Martino, l'altro, con voce lombarda, il _Resegone_, dai molti suoi\n

In [6]:
# ora dobbiamo preparare un prompt per GPT per generare il riassunto del video
# Usiamo il prompt template di langchain
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
	("user", "Riassumi in italiano il seguente testo: {text}")
])

# valorizziamo il template con il contenuto del documento
template = prompt_template.invoke({"text": documents[0].page_content})
print(template)

messages=[HumanMessage(content="Riassumi in italiano il seguente testo: I PROMESSI SPOSI\n                            CAPITOLO PRIMO.\nQuel ramo del lago di Como, che volge a mezzogiorno, tra due catene\nnon interrotte di monti, tutte a seni e a golfi, a seconda dello\nsporgere e del rientrare di quelli, vien, quasi a un tratto, a\nristringersi, e a prender corso e figura di fiume, tra un promontorio\na destra, e un'ampia costiera dall'altra parte; e il ponte, che ivi\ncongiunge le due rive, par che renda ancor più sensibile all'occhio\nquesta trasformazione, e segni il punto in cui il lago cessa, e\nl'Adda rincomincia, per ripigliar poi nome di lago dove le rive,\nallontanandosi di nuovo, lascian l'acqua distendersi e rallentarsi in\nnuovi golfi e in nuovi seni. La costiera, formata dal deposito di tre\ngrossi torrenti, scende appoggiata a due monti contigui, l'uno detto di\nsan Martino, l'altro, con voce lombarda, il _Resegone_, dai molti suoi\ncocuzzoli in fila, che in vero lo fanno

In [7]:
# Ora possiamo invocare GPT per generare il riassunto.
# Avremo bisogno di una API key per OpenAI che verra' presa dall'environment
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# Invochiamo il modello con il template
answer = llm.invoke(template)

# Stampiamo il risultato
print(answer.content)

Nel primo capitolo de "I Promessi Sposi", l'autore descrive il paesaggio attorno al ramo del lago di Como, evidenziando la bellezza naturale della zona, caratterizzata da monti, golfi e torrenti. Il lago si trasforma in fiume, segnato da un ponte che unisce le due rive. La descrizione si sofferma su Lecco, un borgo che si affaccia sul lago e che, ai tempi della narrazione, era già un castello con una guarnigione di soldati spagnoli. Si parla delle strade che collegano le diverse terre, delle vigne e dei boschi, creando un quadro vivace e dettagliato del territorio. Infine, si introduce il personaggio di don Abbondio, curato di una delle terre descritte, che torna a casa dopo una passeggiata, immerso nei suoi pensieri e nella contemplazione del paesaggio circostante.
